In [116]:
import pandas as pd
import pandas_datareader
from pandas_datareader import get_data_yahoo as yh
import itertools

stock_count=5
asset_count=2
ma_short=15
ma_long=50
pv_period=5

In [117]:
myassetlist = [
    'TSLA',
    'GOOG',
    'AAPL',
    'SPY',
    'XLI'
]

In [118]:
df = yh(myassetlist)['Close'] # Get close prices

# Get 20 day returns and rank
ranked_assets = pd.DataFrame(df.pct_change(20).mean().sort_values(ascending = False)) # Ascending to descending returns
ranked_assets # full assets

,0
Symbols,
TSLA,0.068991
AAPL,0.031464
GOOG,0.017376
SPY,0.012283
XLI,0.010961


In [119]:
#moving average short/long term
ranked_ma_assets = pd.DataFrame(((df.tail(ma_short).mean()-df.tail(ma_long).mean())/df.tail(ma_long).mean()).sort_values(ascending = False))
ranked_ma_to_trade = pd.concat([ranked_ma_assets.head(asset_count),ranked_ma_assets.tail(asset_count)])
ranked_ma_to_trade['Direction']= -1
for row in ranked_ma_to_trade.head(asset_count):
    ranked_ma_to_trade.head(asset_count)['Direction']= 1
ranked_ma_to_trade.columns = ['MA_strategy','Direction']
ranked_ma_to_trade

C:\Users\Clara\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,MA_strategy,Direction
Symbols,,
TSLA,0.227206,1
AAPL,0.049673,1
GOOG,0.022687,-1
XLI,0.001150,-1


In [120]:
#Price ratio
#list(itertools.combinations(myassetlist, 2))

In [127]:
#VPT indicator
vf=yh(myassetlist)['Volume']
ranked_pv_assets = pd.DataFrame(((vf*df.pct_change()).cumsum()).pct_change(pv_period).tail(1).sum().sort_values(ascending = False))
ranked_pv_to_trade = pd.concat([ranked_pv_assets.head(asset_count),ranked_pv_assets.tail(asset_count)])
ranked_pv_to_trade['Direction']= -1
for row in ranked_pv_to_trade.head(asset_count):
    ranked_pv_to_trade.head(asset_count)['Direction']= 1
ranked_pv_to_trade.columns = ['VPT_strategy','Direction']
ranked_pv_to_trade

C:\Users\Clara\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,VPT_strategy,Direction
Symbols,,
GOOG,0.147191,1
XLI,0.146917,1
TSLA,-0.004542,-1
AAPL,-0.050381,-1


In [10]:
long = ranked_assets[0:asset_count] # top 2 assets
long['Direction'] = 1
long.columns = ['20_day_returns','Direction']
long

C:\Users\Clara\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,20_day_returns,Direction
Symbols,,
TSLA,0.068216,1
AAPL,0.031434,1


In [11]:
short = ranked_assets[-asset_count:] # bottom 2 assets
short['Direction'] = -1
short.columns = ['20_day_returns','Direction']
short

C:\Users\Clara\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,20_day_returns,Direction
Symbols,,
SPY,0.012303,-1
XLI,0.011206,-1


In [12]:
assets_to_trade = pd.concat([long,short])

In [13]:
assets_to_trade

,20_day_returns,Direction
Symbols,,
TSLA,0.068216,1
AAPL,0.031434,1
SPY,0.012303,-1
XLI,0.011206,-1


# Combine strategies

In [17]:
combined = pd.DataFrame(pd.concat([assets_to_trade,ranked_ma_to_trade,ranked_pv_to_trade])['Direction'])
combined = combined.reset_index()
combined = combined.groupby('Symbols').sum()

C:\Users\Clara\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


TypeError: sort_values() missing 1 required positional argument: 'by'

In [15]:
combined

,Direction
Symbols,
AAPL,2
GOOG,-1
SPY,-1
TSLA,2
XLI,-2
